# 2022 Day 11

https://adventofcode.com/2022/day/11

https://adventofcode.com/2022/day/11/input

In [1]:
import re
from weakref import proxy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
inp = open('input-11.txt').read()

In [3]:
test_inp = """
Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1
""".strip()

In [4]:
def parse_monkey(para):
    # name each line
    id, items, operation, divisor, true_target, false_target = para.split('\n')
    # parse line contents
    id = int(re.match(r'Monkey (\d+)', id).group(1))
    items = list(map(int, re.match(r'.*items: (.*)$', items).group(1).split(', ')))
    m = re.match(r'.*: new = old (.) (.+)', operation)
    operation = m.group(1), None if m.group(2) == 'old' else int(m.group(2))
    divisor = int(re.match(r'.* by (\d+)', divisor).group(1))
    true_target = int(re.match(r'.* monkey (\d+)', true_target).group(1))
    false_target = int(re.match(r'.* monkey (\d+)', false_target).group(1))
    
    return Monkey(
        id = id,
        items = items,
        operation = operation,
        divisor = divisor,
        true_target = true_target,
        false_target = false_target,
    )


def parse_monkeys(inp, chill=3):
    return MonkeyRing(list(map(parse_monkey, inp.split('\n\n'))), chill=chill)


OPERATORS = {
    '*': '__mul__',
    '/': '__div__',
    '+': '__add__',
    '-': '__sub__',
}


class Monkey:
    
    def __init__(
        self,
        *,
        id,
        items,
        operation,
        divisor,
        true_target,
        false_target,
    ):
        self.id = id
        self.items = list(items)
        self.operation = operation
        self.divisor = divisor
        self.true_target = true_target
        self.false_target = false_target
        self.inspections = 0
        
    def __repr__(self):
        return (
            f'Monkey('
            f'id={self.id}, '
            f'items={self.items}, '
            f'operation={self.operation}, '
            f'divisor={self.divisor}, '
            f'true_target={self.true_target}, '
            f'false_target={self.false_target}, '
            f'inspections={self.inspections}, '
            f')'
        )
    
class MonkeyRing:
    
    def __init__(self, monkeys, chill=3):
        self.monkeys = list(monkeys)
        self.chill = chill
        self.divisor = np.prod([monkey.divisor for monkey in monkeys])
        self.dmonkeys = { monkey.id: monkey for monkey in monkeys }
        for monkey in self.monkeys:
            monkey.true_target_monkey = proxy(self.dmonkeys[monkey.true_target])
            monkey.false_target_monkey = proxy(self.dmonkeys[monkey.false_target])
            
    def round(self, n=1, ):
        for i_round in range(n):
            for monkey in self.monkeys:
                op, n = monkey.operation
                while monkey.items:
                    monkey.inspections += 1
                    item = monkey.items.pop(0)
                    op_value = item if n is None else n
                    new_item = getattr(item, OPERATORS[op])(op_value) // self.chill
                    new_item %= self.divisor
                    test_result = (new_item % monkey.divisor == 0)
                    if test_result:
                        monkey.true_target_monkey.items.append(new_item)
                    else:
                        monkey.false_target_monkey.items.append(new_item)
        return self
    
    @property
    def business(self):
        a, b = sorted(monkey.inspections for monkey in self.monkeys)[-2:]
        return a * b
            
    def __repr__(self):
        out = ['MonkeyRing('] + [
            f'  {monkey}' for monkey in self.monkeys
        ] + [')']
        return '\n'.join(out)

In [5]:
monkeys = parse_monkeys(test_inp)
print(monkeys)
monkeys.round(20)
print(monkeys)
monkeys.business

MonkeyRing(
  Monkey(id=0, items=[79, 98], operation=('*', 19), divisor=23, true_target=2, false_target=3, inspections=0, )
  Monkey(id=1, items=[54, 65, 75, 74], operation=('+', 6), divisor=19, true_target=2, false_target=0, inspections=0, )
  Monkey(id=2, items=[79, 60, 97], operation=('*', None), divisor=13, true_target=1, false_target=3, inspections=0, )
  Monkey(id=3, items=[74], operation=('+', 3), divisor=17, true_target=0, false_target=1, inspections=0, )
)
MonkeyRing(
  Monkey(id=0, items=[10, 12, 14, 26, 34], operation=('*', 19), divisor=23, true_target=2, false_target=3, inspections=101, )
  Monkey(id=1, items=[245, 93, 53, 199, 115], operation=('+', 6), divisor=19, true_target=2, false_target=0, inspections=95, )
  Monkey(id=2, items=[], operation=('*', None), divisor=13, true_target=1, false_target=3, inspections=7, )
  Monkey(id=3, items=[], operation=('+', 3), divisor=17, true_target=0, false_target=1, inspections=105, )
)


10605

In [6]:
monkeys = parse_monkeys(test_inp, chill=1)
print(monkeys)
monkeys.round(10_000)
print(monkeys)
monkeys.business

MonkeyRing(
  Monkey(id=0, items=[79, 98], operation=('*', 19), divisor=23, true_target=2, false_target=3, inspections=0, )
  Monkey(id=1, items=[54, 65, 75, 74], operation=('+', 6), divisor=19, true_target=2, false_target=0, inspections=0, )
  Monkey(id=2, items=[79, 60, 97], operation=('*', None), divisor=13, true_target=1, false_target=3, inspections=0, )
  Monkey(id=3, items=[74], operation=('+', 3), divisor=17, true_target=0, false_target=1, inspections=0, )
)
MonkeyRing(
  Monkey(id=0, items=[63602, 56040, 11941, 10573, 61607], operation=('*', 19), divisor=23, true_target=2, false_target=3, inspections=52166, )
  Monkey(id=1, items=[90861, 86149, 27648, 21340, 76915], operation=('+', 6), divisor=19, true_target=2, false_target=0, inspections=47830, )
  Monkey(id=2, items=[], operation=('*', None), divisor=13, true_target=1, false_target=3, inspections=1938, )
  Monkey(id=3, items=[], operation=('+', 3), divisor=17, true_target=0, false_target=1, inspections=52013, )
)


2713310158

## Part 1

In [7]:
monkeys = parse_monkeys(inp)
print(monkeys)
monkeys.round(20)
print(monkeys)
monkeys.business

MonkeyRing(
  Monkey(id=0, items=[72, 97], operation=('*', 13), divisor=19, true_target=5, false_target=6, inspections=0, )
  Monkey(id=1, items=[55, 70, 90, 74, 95], operation=('*', None), divisor=7, true_target=5, false_target=0, inspections=0, )
  Monkey(id=2, items=[74, 97, 66, 57], operation=('+', 6), divisor=17, true_target=1, false_target=0, inspections=0, )
  Monkey(id=3, items=[86, 54, 53], operation=('+', 2), divisor=13, true_target=1, false_target=2, inspections=0, )
  Monkey(id=4, items=[50, 65, 78, 50, 62, 99], operation=('+', 3), divisor=11, true_target=3, false_target=7, inspections=0, )
  Monkey(id=5, items=[90], operation=('+', 4), divisor=2, true_target=4, false_target=6, inspections=0, )
  Monkey(id=6, items=[88, 92, 63, 94, 96, 82, 53, 53], operation=('+', 8), divisor=5, true_target=4, false_target=7, inspections=0, )
  Monkey(id=7, items=[70, 60, 71, 69, 77, 70, 98], operation=('*', 7), divisor=3, true_target=2, false_target=3, inspections=0, )
)
MonkeyRing(
  Monk

58056

## Part 2

In [8]:
monkeys = parse_monkeys(inp, chill=1)
print(monkeys)
monkeys.round(10_000)
print(monkeys)
monkeys.business

MonkeyRing(
  Monkey(id=0, items=[72, 97], operation=('*', 13), divisor=19, true_target=5, false_target=6, inspections=0, )
  Monkey(id=1, items=[55, 70, 90, 74, 95], operation=('*', None), divisor=7, true_target=5, false_target=0, inspections=0, )
  Monkey(id=2, items=[74, 97, 66, 57], operation=('+', 6), divisor=17, true_target=1, false_target=0, inspections=0, )
  Monkey(id=3, items=[86, 54, 53], operation=('+', 2), divisor=13, true_target=1, false_target=2, inspections=0, )
  Monkey(id=4, items=[50, 65, 78, 50, 62, 99], operation=('+', 3), divisor=11, true_target=3, false_target=7, inspections=0, )
  Monkey(id=5, items=[90], operation=('+', 4), divisor=2, true_target=4, false_target=6, inspections=0, )
  Monkey(id=6, items=[88, 92, 63, 94, 96, 82, 53, 53], operation=('+', 8), divisor=5, true_target=4, false_target=7, inspections=0, )
  Monkey(id=7, items=[70, 60, 71, 69, 77, 70, 98], operation=('*', 7), divisor=3, true_target=2, false_target=3, inspections=0, )
)
MonkeyRing(
  Monk

15048718170